# Sistema de Seguridad (Asistente Multimodal) — Vídeo ➜ Vídeo + Voz

**Objetivo del prototipo**  
Procesar vídeos de cámaras de seguridad (`videos/entrada/*.mp4`) y, tras cada vídeo:

- Generar un **vídeo de salida** con las **personas recuadradas** y, si corresponde, la **caja/paquete recuadrada**.
- Mostrar un **print**:
  - Si se detecta que una persona porta una caja/paquete: `Ha llegado un repartidor a tu casa`.
  - Si no: `Ha llegado alguien desconocido a tu casa`.
- Generar un **audio (TTS)** con un mensaje parecido:
  - Repartidor: “Ha llegado un repartidor a tu domicilio, ¿deseas abrirle?”
  - Desconocido: “Ha llegado alguien desconocido a tu domicilio, ¿deseas abrir?”

**Notas de implementación**
- Detección de personas con **YOLOv8**.
- Detección de “caja/paquete” con **CLIP** aplicado a un recorte por persona (heurística ligera).
- Salidas en `videos/salida/`.


In [49]:
# Setup (instalación de dependencias)
# Nota: si ya tienes un venv con todo, esta celda puede omitirse.

import sys
import subprocess

packages = [
    "ultralytics>=8.0.0",
    "opencv-python",
    "pillow",
    "numpy",
    "transformers>=4.35.0",
    "torch",
    "torchvision",
    "gTTS",
    # Evita errores de compatibilidad (p.ej. tensorflow -> protobuf)
    "protobuf>=5.28.0",
]

def pip_install(pkgs):
    cmd = [sys.executable, "-m", "pip", "install", "-q"] + list(pkgs)
    print("Instalando:", " ".join(pkgs))
    subprocess.check_call(cmd)

pip_install(packages)
print("OK")


Instalando: ultralytics>=8.0.0 opencv-python pillow numpy transformers>=4.35.0 torch torchvision gTTS protobuf>=5.28.0
OK


In [50]:
from __future__ import annotations

# IMPORTANTE: evitar que Transformers intente importar TensorFlow.
# En Windows + Python 3.13 es frecuente que TF/protobuf rompan imports.
import os
os.environ.setdefault("TRANSFORMERS_NO_TF", "1")
os.environ.setdefault("TRANSFORMERS_NO_FLAX", "1")

from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import cv2
import numpy as np
from PIL import Image

import torch
from transformers import CLIPModel, CLIPProcessor
from ultralytics import YOLO

from gtts import gTTS

ROOT = Path.cwd()
VIDEOS_IN = ROOT / "videos" / "entrada"
VIDEOS_OUT = ROOT / "videos" / "salida"
VIDEOS_OUT.mkdir(parents=True, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)
print("Input:", VIDEOS_IN)
print("Output:", VIDEOS_OUT)


device: cuda
Input: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\entrada
Output: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida


In [51]:
# Carga de modelos

# YOLOv8 (detección)
yolo = YOLO("yolov8n.pt")

# CLIP (clasificación binaria sobre la persona)
clip_model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)
clip_model = CLIPModel.from_pretrained(clip_model_name).to(device)

COCO_PERSON_CLASS_ID = 0

# Prompts binarios (reducen falsos positivos respecto a buscar "caja" en un recorte pequeño)
CLIP_LABELS = [
    "una persona sosteniendo un paquete o una caja de cartón",  # positivo
    "una persona sin nada en las manos",                       # negativo
]

CLIP_POSITIVE_INDEX = 0
CLIP_NEGATIVE_INDEX = 1

print("Modelos cargados")


Modelos cargados


In [52]:
@dataclass
class Box:
    x1: int
    y1: int
    x2: int
    y2: int

    def clip(self, w: int, h: int) -> "Box":
        return Box(
            x1=max(0, min(self.x1, w - 1)),
            y1=max(0, min(self.y1, h - 1)),
            x2=max(0, min(self.x2, w - 1)),
            y2=max(0, min(self.y2, h - 1)),
        )

    def valid(self) -> bool:
        return self.x2 > self.x1 and self.y2 > self.y1

    def area(self) -> int:
        if not self.valid():
            return 0
        return (self.x2 - self.x1) * (self.y2 - self.y1)


def draw_box(img_bgr: np.ndarray, box: Box, label: str, color: Tuple[int, int, int], thickness: int = 2) -> None:
    cv2.rectangle(img_bgr, (box.x1, box.y1), (box.x2, box.y2), color, thickness)
    if label:
        (tw, th), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        y = max(0, box.y1 - th - baseline - 4)
        cv2.rectangle(img_bgr, (box.x1, y), (box.x1 + tw + 6, y + th + baseline + 6), color, -1)
        cv2.putText(img_bgr, label, (box.x1 + 3, y + th + 3), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)


def tts_to_mp3(text: str, out_path: Path, lang: str = "es") -> Path:
    out_path.parent.mkdir(parents=True, exist_ok=True)
    tts = gTTS(text=text, lang=lang)
    tts.save(str(out_path))
    return out_path


def clip_person_delivery_probs(pil_person: Image.Image) -> Tuple[float, float]:
    """Devuelve (p_pos, p_neg) usando CLIP_LABELS binarios."""
    inputs = clip_processor(text=CLIP_LABELS, images=pil_person, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = clip_model(**inputs)
        probs = outputs.logits_per_image.softmax(dim=1)[0]

    p_pos = float(probs[CLIP_POSITIVE_INDEX].detach().cpu().item())
    p_neg = float(probs[CLIP_NEGATIVE_INDEX].detach().cpu().item())
    return p_pos, p_neg


def person_carry_region(person: Box, frame_w: int, frame_h: int) -> Box:
    # Región auxiliar (solo para visualizar / debug si hace falta)
    w = person.x2 - person.x1
    h = person.y2 - person.y1

    x1 = int(person.x1 - 0.10 * w)
    x2 = int(person.x2 + 0.10 * w)
    y1 = int(person.y1 + 0.15 * h)
    y2 = int(person.y1 + 0.95 * h)

    return Box(x1, y1, x2, y2).clip(frame_w, frame_h)


In [53]:
def process_frame(
    frame_bgr: np.ndarray,
    conf: float = 0.25,
    clip_threshold: float = 0.80,
    clip_margin: float = 0.15,
    min_person_area_ratio: float = 0.03,
) -> Tuple[np.ndarray, bool]:
    """Devuelve (frame_anotado, delivery_detected_en_este_frame).

    Cambios respecto a la versión anterior:
    - CLIP binario sobre el recorte COMPLETO de la persona (menos falsos positivos)
    - Requiere margen p_pos > p_neg + clip_margin
    - Ignora personas muy pequeñas (ruido)
    """
    annotated = frame_bgr.copy()
    h, w = annotated.shape[:2]

    result = yolo.predict(frame_bgr, conf=conf, verbose=False)[0]
    if result.boxes is None or len(result.boxes) == 0:
        return annotated, False

    delivery = False
    min_person_area = int(min_person_area_ratio * (w * h))

    for box_xyxy, cls_id, score in zip(
        result.boxes.xyxy.cpu().numpy(),
        result.boxes.cls.cpu().numpy(),
        result.boxes.conf.cpu().numpy(),
    ):
        if int(cls_id) != COCO_PERSON_CLASS_ID:
            continue

        x1, y1, x2, y2 = [int(v) for v in box_xyxy]
        person = Box(x1, y1, x2, y2).clip(w, h)
        if not person.valid() or person.area() < min_person_area:
            continue

        # Clasificación CLIP sobre el recorte de la persona
        person_crop_bgr = frame_bgr[person.y1 : person.y2, person.x1 : person.x2]
        if person_crop_bgr.size == 0:
            continue

        person_crop_rgb = cv2.cvtColor(person_crop_bgr, cv2.COLOR_BGR2RGB)
        pil_person = Image.fromarray(person_crop_rgb)

        p_pos, p_neg = clip_person_delivery_probs(pil_person)

        # Criterio robusto (reduce falsos positivos)
        is_delivery = (p_pos >= clip_threshold) and (p_pos > p_neg + clip_margin)

        if is_delivery:
            delivery = True
            draw_box(annotated, person, f"REPARTIDOR {score:.2f} | {p_pos:.2f}", (0, 0, 255), 3)
        else:
            draw_box(annotated, person, f"persona {score:.2f} | {p_pos:.2f}", (0, 200, 0), 2)

    return annotated, delivery


In [54]:
def process_video(
    input_path: Path,
    output_path: Path,
    conf: float = 0.25,
    clip_threshold: float = 0.80,
    max_frames: Optional[int] = None,
) -> bool:
    """Procesa un vídeo y devuelve True si se detectó 'repartidor' en algún frame."""

    cap = cv2.VideoCapture(str(input_path))
    if not cap.isOpened():
        raise RuntimeError(f"No se pudo abrir el vídeo: {input_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    if not fps or fps <= 0:
        fps = 25.0

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    output_path.parent.mkdir(parents=True, exist_ok=True)
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))

    delivery_any = False
    frame_idx = 0

    try:
        while True:
            ok, frame = cap.read()
            if not ok:
                break

            annotated, delivery = process_frame(
                frame,
                conf=conf,
                clip_threshold=clip_threshold,
            )
            delivery_any = delivery_any or delivery

            out.write(annotated)

            frame_idx += 1
            if max_frames is not None and frame_idx >= max_frames:
                break

    finally:
        cap.release()
        out.release()

    return delivery_any


In [55]:
def run_on_folder(
    input_dir: Path = VIDEOS_IN,
    output_dir: Path = VIDEOS_OUT,
    conf: float = 0.25,
    clip_threshold: float = 0.35,
    max_frames: Optional[int] = None,
) -> None:
    videos = sorted(input_dir.glob("*.mp4"))
    if not videos:
        print(f"No hay vídeos .mp4 en {input_dir}")
        return

    for in_path in videos:
        out_video = output_dir / f"output_{in_path.name}"
        out_audio = output_dir / f"output_{in_path.stem}.mp3"

        print("\nProcesando:", in_path.name)
        delivery = process_video(
            input_path=in_path,
            output_path=out_video,
            conf=conf,
            clip_threshold=clip_threshold,
            max_frames=max_frames,
        )

        if delivery:
            msg_print = "Ha llegado un repartidor a tu casa"
            msg_tts = "Ha llegado un repartidor a tu domicilio. ¿Deseas abrirle?"
        else:
            msg_print = "Ha llegado alguien desconocido a tu casa"
            msg_tts = "Ha llegado alguien desconocido a tu domicilio. ¿Deseas abrir?"

        print(msg_print)
        tts_to_mp3(msg_tts, out_audio)
        print("Vídeo salida:", out_video)
        print("Audio salida:", out_audio)


# Ejecuta el pipeline sobre todos los vídeos de entrada.
# Consejo: para pruebas rápidas, usa max_frames=150.
run_on_folder(max_frames=None)



Procesando: video_01.mp4
Ha llegado un repartidor a tu casa
Vídeo salida: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida\output_video_01.mp4
Audio salida: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida\output_video_01.mp3

Procesando: video_02.mp4
Ha llegado un repartidor a tu casa
Vídeo salida: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida\output_video_02.mp4
Audio salida: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida\output_video_02.mp3

Procesando: video_03.mp4
Ha llegado un repartidor a tu casa
Vídeo salida: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida\output_video_03.mp4
Audio salida: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida\output_video_03.mp3

Procesando: video_04.mp4
Ha llegado un repartidor a tu casa
Vídeo salida: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida\output_video_04.mp4
Audio salida: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida\output_video_04.mp3

Procesando: video_05.mp4
Ha

In [56]:
def calibrate_clip_threshold_from_negative(
    negative_video: Path,
    conf: float = 0.25,
    clip_margin: float = 0.15,
    sample_every: int = 5,
    max_frames: int = 200,
) -> float:
    """Usa un vídeo NEGATIVO (sin repartidor) para fijar un umbral alto que evite falsos positivos.

    Estrategia: calcula el máximo p_pos observado (con margen p_pos > p_neg + clip_margin) y fija
    threshold = max_p_pos + 0.05 (cap a 0.95).
    """
    cap = cv2.VideoCapture(str(negative_video))
    if not cap.isOpened():
        raise RuntimeError(f"No se pudo abrir el vídeo: {negative_video}")

    max_pos = 0.0
    frame_idx = 0
    used = 0

    try:
        while used < max_frames:
            ok, frame = cap.read()
            if not ok:
                break
            frame_idx += 1
            if sample_every > 1 and (frame_idx % sample_every) != 0:
                continue

            # Detecta personas y mide p_pos (sin escribir vídeo)
            h, w = frame.shape[:2]
            res = yolo.predict(frame, conf=conf, verbose=False)[0]
            if res.boxes is None or len(res.boxes) == 0:
                used += 1
                continue

            for box_xyxy, cls_id in zip(res.boxes.xyxy.cpu().numpy(), res.boxes.cls.cpu().numpy()):
                if int(cls_id) != COCO_PERSON_CLASS_ID:
                    continue
                x1, y1, x2, y2 = [int(v) for v in box_xyxy]
                person = Box(x1, y1, x2, y2).clip(w, h)
                if not person.valid():
                    continue
                crop = frame[person.y1 : person.y2, person.x1 : person.x2]
                if crop.size == 0:
                    continue
                crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
                pil_person = Image.fromarray(crop_rgb)
                p_pos, p_neg = clip_person_delivery_probs(pil_person)
                if p_pos > p_neg + clip_margin:
                    max_pos = max(max_pos, p_pos)

            used += 1

    finally:
        cap.release()

    thr = min(0.95, max(0.70, max_pos + 0.05))
    print(f"Calibración (negativo={negative_video.name}): max_p_pos={max_pos:.3f} -> clip_threshold={thr:.3f}")
    return thr


def run_on_folder(
    input_dir: Path = VIDEOS_IN,
    output_dir: Path = VIDEOS_OUT,
    conf: float = 0.25,
    clip_threshold: Optional[float] = None,
    clip_margin: float = 0.15,
    max_frames: Optional[int] = None,
) -> None:
    videos = sorted(input_dir.glob("*.mp4"))
    if not videos:
        print(f"No hay vídeos .mp4 en {input_dir}")
        return

    # Según tu dataset: el primero NO es repartidor. Lo usamos para calibrar.
    if clip_threshold is None:
        clip_threshold = calibrate_clip_threshold_from_negative(
            negative_video=videos[0],
            conf=conf,
            clip_margin=clip_margin,
            sample_every=5,
            max_frames=200,
        )

    for idx, in_path in enumerate(videos, start=1):
        out_video = output_dir / f"output_{in_path.name}"
        out_audio = output_dir / f"output_{in_path.stem}.mp3"

        print("\nProcesando:", in_path.name)
        delivery = process_video(
            input_path=in_path,
            output_path=out_video,
            conf=conf,
            clip_threshold=clip_threshold,
            max_frames=max_frames,
        )

        if delivery:
            msg_print = "Ha llegado un repartidor a tu casa"
            msg_tts = "Ha llegado un repartidor a tu domicilio. ¿Deseas abrirle?"
        else:
            msg_print = "Ha llegado alguien desconocido a tu casa"
            msg_tts = "Ha llegado alguien desconocido a tu domicilio. ¿Deseas abrir?"

        print(f"[{idx}/{len(videos)}] {msg_print}")
        tts_to_mp3(msg_tts, out_audio)
        print("Vídeo salida:", out_video)
        print("Audio salida:", out_audio)


# Ejecuta el pipeline sobre todos los vídeos de entrada.
run_on_folder(max_frames=None)


Calibración (negativo=video_01.mp4): max_p_pos=0.937 -> clip_threshold=0.950

Procesando: video_01.mp4
[1/5] Ha llegado alguien desconocido a tu casa
Vídeo salida: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida\output_video_01.mp4
Audio salida: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida\output_video_01.mp3

Procesando: video_02.mp4
[2/5] Ha llegado un repartidor a tu casa
Vídeo salida: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida\output_video_02.mp4
Audio salida: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida\output_video_02.mp3

Procesando: video_03.mp4
[3/5] Ha llegado un repartidor a tu casa
Vídeo salida: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida\output_video_03.mp4
Audio salida: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida\output_video_03.mp3

Procesando: video_04.mp4
[4/5] Ha llegado un repartidor a tu casa
Vídeo salida: c:\Users\jordi\Documents\UNI\SegurityGuard\videos\salida\output_video_04.mp4
Audio salida